In [1]:
import os
import json

log_path = "C:/Users/29059/Desktop/CompositionalArena/logs/"
subtask_id = "a7310aa0-b194-77e3-5c36-996391a1bc7d"
round_num = 1
trajectory_path = f"{log_path}{subtask_id}({round_num})/"
actions = []

# Traverse all files ending in 'response.json' in the trajectory path.
for filename in os.listdir(trajectory_path):
    if filename.endswith("response.json"):
        file_path = os.path.join(trajectory_path, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)
        actions.append({"ControlText": data['response']['ControlText'], "Function": data['response']['Function'], "Args": data['response']['Args']})
        print(data['response']['ControlText'])  # File Explorer
        print(data['response']['Function'])  # click_input
        print(data['response']['Args'])  # {'button': 'left', 'double': False}





Excel
click_input
{'button': 'left', 'double': False}
The Evolution of Urbanization Rate.xlsx
click_input
{'button': 'left', 'double': False}
A
click_input
{'button': 'left', 'double': False}
Center
click_input
{'button': 'left', 'double': False}


{}


In [2]:
# utils.py
from rapidfuzz import fuzz

def fuzzy_match(str1, str2, threshold=60):
    similarity = fuzz.ratio(str1, str2)
    if similarity >= threshold:
        return True
    return False

def fuzzy_in(str1, str2, threshold=60):
    len_str1 = len(str1)
    for i in range(len(str2) - len_str1 + 1):
        substring = str2[i:i + len_str1]
        similarity = fuzz.ratio(str1, substring)
        if similarity >= threshold:
            return True
    return False

# str1 = "set as background"
# str2 = "Set as desktop background"
# str3 = "Right-click to set as desktop background of your screen."
# print(fuzzy_match(str1, str2))
# print(fuzzy_in(str1, str3))
# print(fuzzy_in(str2, str3))


In [10]:
# eval_APIs.py
def check_mouse_clicks(text: str) -> bool:
    """Checks if the mouse has clicked on the specified text.
    Parameters
    ---------
    text: str
        The text associated with the click.
    Returns
    ---------
    bool
        True if the mouse has clicked on the specified text, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully set the picture 'envelope.png' as background
    >>> def evaluate_agent_task_completion():
    >>>     if not check_mouse_clicks(text='envelope.png'):
    >>>         return False
    >>>     if not check_mouse_clicks(text='set as background'):
    >>>         return False
    >>>     return True
    """
    for action in actions:
        if (action['Function'] == "click_input" and
            action['Args']['button'] == button and
            action['Args']['double'] == double and
            fuzzy_match(text.upper(), action['ControlText'].upper())):
            return True
    return False

def check_keyboard_types(text: str) -> bool:
    """Checks if the keyboard has typed the specified text.
    Parameters
    ---------
    text: str
        The text to be typed.
    Returns
    ---------
    bool
        True if the keyboard has typed the specified text, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully typed 'Hello, World!'
    >>> def evaluate_agent_task_completion():
    >>>     if not check_keyboard_types(text='Hello, World!'):
    >>>         return False
    >>>     return True
    """
    for action in actions:
        if (action['Function'] == "keyboard_input" and
            fuzzy_match(text.upper(), action['Args']['keys'].upper())):
            return True
    return False

# 待修改，现在是检查本地文件是否存在，应该改成检查虚拟机文件是否存在
def check_file_exists(file_path: str) -> bool:
    """Checks if the specified file exists.
    Parameters
    ---------
    file_path: str
        The path to the file to be checked.
    Returns
    ---------
    bool
        True if the file exists, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully renamed 'cat.jpg' to 'cute cat.jpg'
    >>> def evaluate_agent_task_completion():
    >>>     if check_file_exists(file_path='C:/Users/user/Desktop/images/cat.jpg'):
    >>>         return False
    >>>     if not check_file_exists(file_path='C:/Users/user/Desktop/images/cute cat.jpg'):
    >>>         return False
    >>>     return True
    """
    import os
    return os.path.exists(file_path)

def check_text_exists_via_ocr(text: str) -> bool:
    """Checks if the specified text is present in the last screenshot using OCR (Optical Character Recognition).
    Parameters
    ---------
    text: str
        The text to be checked.
    Returns
    ---------
    bool
        True if the text is present in the last screenshot, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully set the clock to '9:00 AM'
    >>> def evaluate_agent_task_completion():
    >>>     if not check_text_exists_via_ocr(text='9:00 AM'):
    >>>         return False
    >>>     return True
    """
    import easyocr
    step_num = len(actions)
    last_screenshot_path = os.path.join(trajectory_path, f"action_step{step_num-1}.png")
    reader = easyocr.Reader(['en'])
    ocr_result = reader.readtext(last_screenshot_path, detail=0)
    ocr_result = ' '.join(ocr_result)
    return fuzzy_in(text.upper(), ocr_result.upper())

def check_text_exists_via_control(text: str) -> bool:
    """Checks if the specified text is present in the last screenshot through control information.
    Parameters
    ---------
    text: str
        The text to be checked.
    Returns
    ---------
    bool
        True if the text is present in the last screenshot, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully input the code 'print("Hello World!")'
    >>> def evaluate_agent_task_completion():
    >>>     if not check_text_exists_via_control(text='print("Hello World!")'):
    >>>         return False
    >>>     return True
    """
    step_num = len(actions)
    last_control_path = os.path.join(trajectory_path, f"action_step{step_num-1}_controls_info.json")
    with open(last_control_path, 'r') as f:
        control_info = json.load(f)
    control_titles = [control['title'] for control in control_info]
    control_titles = ' '.join(control_titles)
    return fuzzy_in(text.upper(), control_titles.upper())

def check_text_exists(text: str) -> bool:
    """Checks if the specified text is included in the last screenshot.
    Parameters
    ---------
    text: str
        The text to be checked.
    Returns
    ---------
    bool
        True if the text is present in the last screenshot, False otherwise.
    Examples
    ---------
    >>> # Evaluate if the agent has successfully created a new folder named 'Project Files'
    >>> def evaluate_agent_task_completion():
    >>>     if not check_text_exists(text='Project Files'):
    >>>         return False
    >>>     return True
    """
    return check_text_exists_via_ocr(text) or check_content_exists_via_controls(content)

# print(check_content_exists_via_control("Close Tab"))
# print(check_content_exists_via_ocr("share"))


In [12]:
def evaluate_agent_task_completion():
    # Check if the file 'The Evolution of Urbanization Rate.xlsx' was opened
    if not check_file_exists(file_path='C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx'):
        return False

    # Check if the 'A' column was selected
    if not check_text_exists_via_control(text='A'):
        return False

    # Check if the content was centered
    if not check_text_exists_via_control(text='Center'):
        return False

    # All checks passed, task is considered complete
    return True



# Explanation:
# 1. **File Existence**: Confirms that 'envelope.png' exists.
# 2. **Selection Verification**: Uses a hypothetical UI element (like selection highlighting) to confirm selection.
# 3. **Rotation Action**: Checks for the click action associated with rotating the image.
# 4. **Rotation Confirmation**: Uses OCR to confirm any label or visual hint proving rotation.
print(evaluate_agent_task_completion())

True


In [21]:
import pytesseract 
import cv2
from PIL import Image 
# 英文识别：将路径改为自己要识别的图片路径即可 
# 读取图像
image = cv2.imread(r"C:\Users\29059\Desktop\CompositionalArena\logs\933f7b16-8e71-042b-1629-4027dd2a3f15(0)\action_step7_selected_controls.png")
custom_config = r'-c tessedit_char_whitelist= .abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
# # 灰度化
# image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # 去噪
# image = cv2.medianBlur(image, 3)

# OCR 识别，使用 psm 6 模式
text = pytesseract.image_to_string(image, config=custom_config)
print(text) 

9 items

+

Q > ThisPC > LocalDisk(C) > Users > user > Desktop > images

W Sort» © View »

cliffjpg envelopepng feastjpg sports carjpg

QBMOCKHTSORETRPEEVOXT B#OSCZEBEO

Search images

@ Details

images (9 items)

Select a single file to get more
@ information and share your
cloud content.

10:02 AM
Q

a & wns Cae sne/a0z4



In [24]:
import easyocr

reader = easyocr.Reader(['en'])
result = reader.readtext(r"C:\Users\29059\Desktop\CompositionalArena\logs\933f7b16-8e71-042b-1629-4027dd2a3f15(0)\action_step7_selected_controls.png", detail=0)
for text in result:
    print(text)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


images
This PC
Local Disk (C:)
Users
user
Desktop
images
Search images
New
Sort
View
Details
Cen
beachjpg
catjpg
cityjpg
dogjpg
envelope png
feastjpg
sports
sunjpg
images (9 items)
Select a single file to
more
information and share your
cloud content
items
10.02 AM
P
ENG
C 4x
8/16/2024
cliffjpg
carjpg
get
